# Exploration

The dataset I'm using came from Mountain Project. Filters include:

- Location: Boulder Canyon
- Type: 
    - Rock
    - 5.0 to 5.15d (start of technical climbing grade to the max in the grade)
    - Trad, Sport, and Toprope
- No quality or pitch filters

In [1]:
# Import modules
import os, sys, re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Get path
data_dir = os.path.realpath('../data/')

## Data Cleaning

I'll start by getting an overview the data and making it more usable.

In [2]:
# Upload data
df_mp = pd.read_csv(data_dir + '/boulder_sport_50_515d.csv', header=0,
                    sep=',')
print('Frame shape: ' + str(df_mp.shape))
df_mp.tail(2)

Frame shape: (945, 11)


,Route,Location,URL,Avg Stars,Your Stars,Route Type,Rating,Pitches,Length,Area Latitude,Area Longitude
943,Serps Up,Serpent's Den > Boulder Canyon > Boulder > Col...,https://www.mountainproject.com/route/12226557...,3.0,-1,Sport,5.13b,1,25.0,40.00239,-105.41014
944,Aqueduct,Aqueduct Outcrop > Boulder Canyon > Boulder > ...,https://www.mountainproject.com/route/12246443...,2.0,-1,Sport,5.12a,1,50.0,39.97527,-105.46080


In [3]:
df_mp.dtypes

Route              object
Location           object
URL                object
Avg Stars         float64
Your Stars          int64
Route Type         object
Rating             object
Pitches             int64
Length            float64
Area Latitude     float64
Area Longitude    float64
dtype: object

Variables include route name, location, Mountain Project URL, average stars, my ratings, route type, rating, pitches, length, and geographic information. 

There are a mix of objects (strings), floats, and integers. Something to note is that "Rating" is coming out as an object when it can be a float. This is a transformation I'll do later on.

I'll do some data cleaning to only include useful variables and make variable names easier to work with. 

In [4]:
# Copy original frame
df_copy = df_mp.copy()

# Simplify variable names
df_copy.columns = [x.lower().replace(' ', '_') for x in list(df_copy.columns)]

# Remove my ratings (your_stars)
df_copy = df_copy.drop('your_stars', 1)
df_copy.tail(2)

,route,location,url,avg_stars,route_type,rating,pitches,length,area_latitude,area_longitude
943,Serps Up,Serpent's Den > Boulder Canyon > Boulder > Col...,https://www.mountainproject.com/route/12226557...,3.0,Sport,5.13b,1,25.0,40.00239,-105.41014
944,Aqueduct,Aqueduct Outcrop > Boulder Canyon > Boulder > ...,https://www.mountainproject.com/route/12246443...,2.0,Sport,5.12a,1,50.0,39.97527,-105.46080


I'll check for null and null replacement values next. Based on the "Your Stars" rating from the original frame, it looks like -1 is used as a replacement value.

In [5]:
# Loop through columns to find -1
for curr_col in list(df_copy.columns):
    
    search_null = df_copy[df_copy[curr_col] == -1].shape[0]
    
    if search_null != 0:
        print(f'{curr_col}: {search_null} rows with -1' )

avg_stars: 7 rows with -1
pitches: 1 rows with -1


I double checked and did see 7 rows with -1 in "avg_stars" and 1 row with -1 in "pitches." It's not possible to have negative stars or pitches, so these must be value substitutes. I'll change these to nulls to not confuse nulls from actual values.

In [6]:
# Replace -1 with null
df_copy = df_copy.replace(to_replace=-1, value=np.nan)  # np.nan for numerical

In [7]:
# Count remaining null values d
df_copy.isna().sum()

route               0
location            0
url                 0
avg_stars           7
route_type          0
rating              0
pitches             1
length            197
area_latitude       0
area_longitude      0
dtype: int64

In addition to the 8 placeholder values that were changed to nulls, there are 197 missing length values.

The next thing I want to do here is to change ratings from objects to floats. All of the climbs are in the 5.0-5.15 range because I'm looking at technical climbing. I'll remove the 5 and turn the string portion of the rating (e.g. a, b, c, d) to numbers.

Ratings are full of information. Generally, once the grade is at or above 5.10, the sub-classifications break out into a, b, c, and d. However, there are other climbers who use + and - for sub-classifications. Climbs below 5.10 typically use + and - for sub-classifications. I'll convert alphabetical sub-classifications into a numberical rating in the tenths place. I'll then convert + and - to fall within the tenths numerical ratings.

Some routes contain boldering grades (e.g. VX), which will be removed.

Protection ratings should be included in the rating if the route is not well bolted. If a protection rating isn't included (e.g. PG/PG13, R), the reader can assume that the route is well bolted (i.e. G).

In [8]:
# Dictionary for cleaning sub-classifications
['', '+', '-', 'a', 'a/b', 'b', 'b/c', 'c', 'c/d', 'd']

sub_classification_dict = {
    '' : 0,
    '+' : 35,
    '-' : 15,
    'a' : 1,
    'a/b' : 15,
    'b' : 2,
    'b/c' : 25,
    'c' : 3,
    'c/d' : 35,
    'd' : 4
}

# Dictionary for cleaning protection ratings
protection_dict = {
    'A0' : 'G',
    'G' : 'G',
    'PG13' : 'PG13',
    'R' : 'R',
    'V10' : 'G',
    'V6' : 'G',
    'V7' : 'G',
    'X' : 'G'
}

In [9]:
# Climbing rating transformation
# Remove technical climbing int
df_copy['climbing_rating'] = df_copy['rating'].apply(
    lambda x: x.replace('5.', '').split(' ')
)

# Get main rating
df_copy['climbing_rating_main'] = df_copy['climbing_rating'] \
    .apply(lambda x: re.match(r'(\d+)', x[0]).group(1))
df_copy[['climbing_rating', 'climbing_rating_main']].sample(2)

,climbing_rating,climbing_rating_main
372,[11b/c],11
305,[9],9


In [10]:
# Find sub-classification
df_copy['climbing_rating_sub'] = df_copy['climbing_rating'] \
    .apply(lambda x: (re.match(r'\d+(.*)', x[0]).group(1)))
print(sorted(list(set(list(df_copy['climbing_rating_sub'])))))

# Clean sub-classification
df_copy['climbing_rating_sub_clean'] = df_copy['climbing_rating_sub'] \
    .map(sub_classification_dict)
df_copy['climbing_rating_sub_clean'] = df_copy['climbing_rating_sub_clean'].fillna(0)
df_copy['climbing_rating_sub_clean'] = df_copy['climbing_rating_sub_clean'].astype(int)
df_copy['climbing_rating_complete'] = df_copy \
    .apply(lambda x: float(str(x['climbing_rating_main']) + '.' + str(x['climbing_rating_sub_clean'])), 1)
df_copy[['climbing_rating', 'climbing_rating_complete']].sample(2)

['', '+', '-', 'a', 'a/b', 'b', 'b/c', 'c', 'c/d', 'd']


,climbing_rating,climbing_rating_complete
699,[11a],11.1
714,[11d],11.4


In [11]:
# Get protection value
df_copy['protection'] = df_copy['climbing_rating'] \
    .apply(lambda x: [x[1] if len(x) > 1 else 'G'][0])
print(sorted((list(set(list(df_copy['protection']))))))

# Clean protection
df_copy['protection'] = df_copy['protection'].map(protection_dict)
df_copy[['rating', 'climbing_rating_complete', 'protection']].sample(2)

['A0', 'G', 'PG13', 'R', 'V10', 'V6', 'V7', 'X']


,rating,climbing_rating_complete,protection
240,5.12b/c,12.25,G
178,5.12a,12.10,G


In [14]:
# Check work
climbing_ratingls = sorted(list(set(list(df_copy['climbing_rating_complete']))))
protectionls = sorted(list(set(list(df_copy['protection']))))
print('Shape: ' + str(df_copy.shape))
print('Climbing ratings: ' + str(climbing_ratingls))
print('Protection ratings: ' + str(protectionls))

Shape: (945, 16)
Climbing ratings: [3.0, 4.0, 5.0, 6.0, 7.0, 7.35, 8.0, 8.15, 8.35, 9.0, 9.15, 9.35, 10.0, 10.1, 10.15, 10.2, 10.25, 10.3, 10.35, 10.4, 11.0, 11.1, 11.15, 11.2, 11.25, 11.3, 11.35, 11.4, 12.0, 12.1, 12.15, 12.2, 12.25, 12.3, 12.35, 12.4, 13.0, 13.1, 13.15, 13.2, 13.25, 13.3, 13.4, 14.1, 14.3]
Protection ratings: ['G', 'PG13', 'R']


Now that the ratings are complete, I'll save the completed frame and move onto the exploration piece.

In [15]:
# Save frame
drop_colls = ['climbing_rating', 'climbing_rating_main',
              'climbing_rating_sub', 'climbing_rating_sub_clean']
df_save = df_copy.drop(drop_colls, 1)
df_save.to_csv(data_dir + '/df_clean.csv', index=False)
df_save.sample(2)

,route,location,url,avg_stars,route_type,rating,pitches,length,area_latitude,area_longitude,climbing_rating_complete,protection
516,Double Impact,Frisky Cliff > Boulder Canyon > Boulder > Colo...,https://www.mountainproject.com/route/11100117...,2.8,Sport,5.13b,1.0,45.0,39.97727,-105.45706,13.20,G
50,Wimps,Nip and Tuck > Boulder Canyon > Boulder > Colo...,https://www.mountainproject.com/route/11121692...,1.8,Sport,5.8+,1.0,45.0,39.98709,-105.43728,8.35,G
